In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
cv2.__version__

'4.4.0'

In [3]:
# имена папок. они же - имена классов.
labels=['karabin','hryvnia','stepler','trash']
#извлекаем списки файлов для каждой папки.
filelists={}
for i in labels:
    filelists[i]=os.listdir(i)

# Сейчас надо схлопнуть один большой дескриптор (nx64 для BRISK) из всех изображений.

In [4]:
orb=cv2.ORB_create()
sift=cv2.SIFT_create(nfeatures=500)
brisk=cv2.BRISK_create()
estimator=brisk
name_estimator="brisk"

#SIFT очень долго выполняет это место(~30 минут.)
des_list=[]
des_dict={}
failed_files=[] #файлы, для которых des=None
for i in labels:
    for j in filelists[i]:
        pathname=f'{i}/{j}'
        img=cv2.imread(pathname,0)
        img=cv2.resize( img, (img.shape[0]//4,img.shape[1]//4) )
        kp,des=estimator.detectAndCompute(img,None)
        if(des is None):
            failed_files.append(pathname)
            continue
        des=des.astype('float32')
        if(des.shape[0]>500):
            des=des[:500,:]
        des_list.append(des)
        des_dict[pathname]=des
        print(pathname, des.shape)

karabin/(69).jpg (255, 64)
karabin/(73).jpg (500, 64)
karabin/(60).jpg (500, 64)
karabin/(23).jpg (203, 64)
karabin/(67).jpg (500, 64)
karabin/(25).jpg (500, 64)
karabin/(71).jpg (500, 64)
karabin/(82).jpg (500, 64)
karabin/(38).jpg (500, 64)
karabin/(1).jpg (500, 64)
karabin/(79).jpg (500, 64)
karabin/(40).jpg (500, 64)
karabin/(4).jpg (500, 64)
karabin/(63).jpg (500, 64)
karabin/(64).jpg (500, 64)
karabin/(76).jpg (500, 64)
karabin/(39).jpg (500, 64)
karabin/(83).jpg (500, 64)
karabin/(95).jpg (500, 64)
karabin/(34).jpg (500, 64)
karabin/(29).jpg (8, 64)
karabin/(12).jpg (500, 64)
karabin/(17).jpg (500, 64)
karabin/(14).jpg (500, 64)
karabin/(48).jpg (500, 64)
karabin/(35).jpg (500, 64)
karabin/(45).jpg (500, 64)
karabin/(5).jpg (500, 64)
karabin/(74).jpg (500, 64)
karabin/(18).jpg (500, 64)
karabin/(87).jpg (500, 64)
karabin/(51).jpg (500, 64)
karabin/(56).jpg (500, 64)
karabin/(19).jpg (500, 64)
karabin/(81).jpg (500, 64)
karabin/(99).jpg (500, 64)
karabin/(20).jpg (500, 64)
karabi

stepler/(29).jpg (500, 64)
stepler/(12).jpg (500, 64)
stepler/KIMG1122.jpg (500, 64)
stepler/(17).jpg (500, 64)
stepler/(14).jpg (500, 64)
stepler/(48).jpg (366, 64)
stepler/(35).jpg (500, 64)
stepler/(45).jpg (178, 64)
stepler/KIMG1125.jpg (500, 64)
stepler/KIMG1101.jpg (500, 64)
stepler/(5).jpg (57, 64)
stepler/KIMG1094.jpg (500, 64)
stepler/(74).jpg (500, 64)
stepler/KIMG1093.jpg (500, 64)
stepler/(18).jpg (500, 64)
stepler/(87).jpg (500, 64)
stepler/(51).jpg (500, 64)
stepler/(56).jpg (500, 64)
stepler/(19).jpg (500, 64)
stepler/(81).jpg (500, 64)
stepler/(99).jpg (500, 64)
stepler/(20).jpg (500, 64)
stepler/(15).jpg (500, 64)
stepler/(10).jpg (500, 64)
stepler/KIMG1118.jpg (500, 64)
stepler/(37).jpg (500, 64)
stepler/(80).jpg (235, 64)
stepler/KIMG1106.jpg (422, 64)
stepler/KIMG1098.jpg (500, 64)
stepler/(33).jpg (500, 64)
stepler/KIMG1119.jpg (154, 64)
stepler/(77).jpg (88, 64)
stepler/(42).jpg (295, 64)
stepler/(47).jpg (137, 64)
stepler/KIMG1108.jpg (500, 64)
stepler/KIMG1124.j

In [ ]:
#тяжелая ячейка, комп начнет гудеть
from sklearn.cluster import KMeans
X=np.vstack(des_list)
kmeans=KMeans(n_clusters=10*4, random_state=0).fit( X )
print(kmeans.labels_)

In [ ]:
def create_bag(labels_list):
    res=np.zeros((10*4,))
    for i in labels_list:
        res[i]+=1
    return res
#Как работает bag-of words? Допустим, уникальные элементы - 0,1,2,3,4.
#Тогда [0,2,0,2,4,4,4,3,3,0,1] мы посчитаем частоты каждого ->[3,1,2,2,3]. 
#То что получилось - это 5 фичей, на основе которых будем проводить классификацию.
list_of_dicts=[]
for i in labels:
    for j in filelists[i]:
        pathname=f'{i}/{j}'
        if(pathname in failed_files):
            continue
        des=des_dict[pathname]
        kmeans_labels=kmeans.predict(des)
        bag_of_words=create_bag(kmeans_labels)
        temp_dict={}
        temp_dict['pathname']=pathname
        for k in np.arange(10*4):
            temp_dict[f'feature_{k}']=bag_of_words[k]
        temp_dict['target']=i
        list_of_dicts.append(temp_dict)

df=pd.DataFrame(list_of_dicts)
df.head()
df.to_csv(f"{name_estimator}_dataset.csv",index=False)

In [ ]:
from joblib import dump, load
dump(kmeans, f'written_models/{name_estimator}_kmeans.joblib') 